In [ ]:
## docs
# appstoreconnect - https://pypi.org/project/appstoreconnect/
# api - https://developer.apple.com/documentation/appstoreconnectapi

In [ ]:
# instalando libs
!pip install appstoreconnect
!pip install google-cloud-secret-manager
!pip install pandas-gbq

In [ ]:
# importando libs
import pandas as pd
import pandas_gbq

from datetime import datetime, timedelta
from appstoreconnect import Api, UserRole
from google.cloud import secretmanager

In [ ]:
# declarando função para consuno do KMS (KeyManager no GCP)

def keygcp(secret_id, project_id="dev-stalse"):
    """ 
    Função para consumir as chaves secretas que estão cadastradas no GCP
    
    - secret_id é o nome da chave, exemplo: msc-issuerid-apple
                                             |       |     |
                                             |       |     |--> API
                                             |       |
                                             |       |--> variavel desejada
                                             |
                                             |--> abreviação do nome do cliente (Mosaic = msc)

    - projetct_id está por default o projeto "dev-stalse", mas se for necessário poderá ser alterado na chamada da função
    """
    
    # declarando variaveis para uso na API KMS
    _secret_id = secret_id
    _project_id = project_id
    
    # Incializando client KMS
    secretmanager_client = secretmanager.SecretManagerServiceClient()

    # requisitando KMS
    response=secretmanager_client.access_secret_version(
        name=f'projects/{_project_id}/secrets/{_secret_id}/versions/latest' # ultima versao da key
    )

    # retornando valor da secret
    key = response.payload.data.decode("UTF-8")
    return key

In [ ]:
# consumindo chaves do KMS para usar na chamada da API e do report
issuer_id = keygcp('msc-issuerid-apple') # id único da Mosaic dentro da plataforma Apple
key_id = keygcp('msc-keyid-apple') # id único do usuário dados@stalse
path_to_key_file = keygcp('msc-privatekey-apple') # chave privada criada a partir do key_id
vendor_number = keygcp('msc-vendor-apple') # id da mosaic para criar o relatório 'download_sales_and_trends_reports'

In [ ]:
# criando client do appstoreconnect com as variáveis do KMS
api = Api(key_id, path_to_key_file, issuer_id)

In [ ]:
# filtrando data para processar sempre d-1
hoje = datetime.now() # data de hoje
ontem = hoje - timedelta(1) # calculando d-1
ontem = ontem.strftime('%Y-%m-%d') # formatando data(d-1) para usar na váriavel 'reportDate'

# validando se a data(d-1) contém informação
try:
    # realizando o 'download sales and trends reports'
    report = api.download_sales_and_trends_reports(
    filters={'frequency': 'DAILY',
             'reportSubType': 'SUMMARY',
             'reportType': 'SALES',
             'vendorNumber': vendor_number,
             'reportDate': ontem}
    ,save_to='report.csv')
    
    # lendo o csv para crianção do df
    csv = pd.read_csv("report.csv", encoding='utf-8', sep='\t', low_memory=False, on_bad_lines='skip', skipinitialspace=True)
    # criando df
    df_report = pd.DataFrame(data=csv)
    print('Processado os dados do dia {}'.format(ontem))

# se não conter informação no report
except:
    print('Não houve dados para o dia {}'.format(ontem))

In [ ]:
# validando se o csv contém informação
try:
    # criando csv para colocar os dados no df
    df_report.to_csv("report.csv", sep=";", index=False)

    csv = pd.read_csv("report.csv", encoding='utf-8', sep=';', low_memory=False, on_bad_lines='skip', skipinitialspace=True)
    
    # renomeando colunas incompátiveis com o bq
    csv.rename(columns={'Provider Country': 'Provider_Country', 
                'Product Type Identifier': 'Product_Type_Identifier', 
                'Developer Proceeds': 'Developer_Proceeds',
                'Begin Date': 'Begin_Date',
                'End Date': 'End_Date',
                'Customer Currency': 'Customer_Currency',
                'Country Code': 'Country_Code',
                'Currency of Proceeds': 'Currency_of_Proceeds',
                'Apple Identifier': 'Apple_Identifier',
                'Customer Price': 'Customer_Price',
                'Promo Code': 'Promo_Code',
                'Parent Identifier': 'Parent_Identifier',
                'Supported Platforms': 'Supported_Platforms',
                'Proceeds Reason': 'Proceeds_Reason',
                'Preserved Pricing': 'Preserved_Pricing',
                'Order Type': 'Order_Type'}, inplace=True)
    
# se não conter informação
except:
    print('Não houve dados para o dia {}'.format(ontem))

In [ ]:
# inserindo os dados no bq
print('Pronto para inserir os dados no BigQuery.')

# validando se o csv contém informação
try:
    print('Inserindo {} linhas a tabela de dados agregados.'.format(len(csv)))

    pandas_gbq.to_gbq(csv, 
                        'nutrisafras_apple.app_installations',
                        project_id='mosaic-fertilizantes',
                        if_exists='append',
                        table_schema=[
                        {'name': 'Provider','type': 'STRING'},
                        {'name': 'Provider_Country','type': 'STRING'},
                        {'name': 'SKU','type': 'STRING'},
                        {'name': 'Developer','type': 'STRING'},
                        {'name': 'Title','type': 'STRING'},
                        {'name': 'Version','type': 'STRING'},
                        {'name': 'Product_Type_Identifier','type': 'STRING'},
                        {'name': 'Units','type': 'INTEGER'},
                        {'name': 'Developer_Proceeds','type': 'FLOAT'},
                        {'name': 'Begin_Date','type': 'STRING'},
                        {'name': 'End_Date','type': 'STRING'},
                        {'name': 'Customer_Currency','type': 'STRING'},
                        {'name': 'Country_Code','type': 'STRING'},
                        {'name': 'Currency_of_Proceeds','type': 'STRING'},
                        {'name': 'Apple_Identifier','type': 'INTEGER'},
                        {'name': 'Customer_Price','type': 'FLOAT'},
                        {'name': 'Promo_Code','type': 'FLOAT'},
                        {'name': 'Parent_Identifier','type': 'FLOAT'},
                        {'name': 'Subscription','type': 'FLOAT'},
                        {'name': 'Period','type': 'FLOAT'},
                        {'name': 'Category','type': 'STRING'},
                        {'name': 'CMB','type': 'FLOAT'},
                        {'name': 'Device','type': 'STRING'},
                        {'name': 'Supported_Platforms','type': 'STRING'},
                        {'name': 'Proceeds_Reason','type': 'FLOAT'},
                        {'name': 'Preserved_Pricing','type': 'FLOAT'},
                        {'name': 'Client','type': 'FLOAT'},
                        {'name': 'Order_Type','type': 'FLOAT'}
                        ],
                        location='us-east4',
                        progress_bar=True,
                        api_method='load_csv')

    print('Todas as {} linhas foram inseridas na tabela.'.format(len(csv)))
        
    print('Script executado com sucesso.')

# se não conter informação   
except:
    print('Não houve dados para o dia {}'.format(ontem))